# Modeling

In [4]:
import os

import pandas as pd
import numpy as np

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, cross_val_score, RandomizedSearchCV, validation_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, classification_report


from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


from sklearn.feature_selection import mutual_info_regression, SelectKBest, f_regression
from sklearn.utils import resample, shuffle

from sklearn.preprocessing import LabelEncoder

random_state=42

In [5]:
# To suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [7]:
# df= pd.read_csv('/Users/sayalinagarkar/Documents/Sayali/INFO-6105/Project/US_1921_2021_normalized.csv')

df= pd.read_csv('/Users/sayalinagarkar/Documents/Sayali/INFO-6105/Project/DataSci_Project/US_1921_2021_normalized.csv')

In [8]:
df.shape

(587927, 15)

In [9]:
df.head()

,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration
0,0.06,1922,0.645,0.4450,C,-13.338,major,0.4510,0.674,0.7440,0.151,0.127,104.851,3,126.903
1,0.00,1922,0.695,0.2630,C,-22.136,major,0.9570,0.797,0.0000,0.148,0.655,102.009,1,98.200
2,0.00,1922,0.434,0.1770,C#,-21.180,major,0.0512,0.994,0.0218,0.212,0.457,130.418,5,181.640
3,0.00,1922,0.321,0.0946,G,-27.961,major,0.0504,0.995,0.9180,0.104,0.397,169.980,3,176.907
4,0.00,1922,0.402,0.1580,D#,-16.900,minor,0.0390,0.989,0.1300,0.311,0.196,103.220,4,163.080


We won't need years so dropping it.

In [10]:
df_no_year= df.drop('year', axis=1)

In [11]:
df_no_year.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration
0,0.06,0.645,0.4450,C,-13.338,major,0.4510,0.674,0.7440,0.151,0.127,104.851,3,126.903
1,0.00,0.695,0.2630,C,-22.136,major,0.9570,0.797,0.0000,0.148,0.655,102.009,1,98.200
2,0.00,0.434,0.1770,C#,-21.180,major,0.0512,0.994,0.0218,0.212,0.457,130.418,5,181.640
3,0.00,0.321,0.0946,G,-27.961,major,0.0504,0.995,0.9180,0.104,0.397,169.980,3,176.907
4,0.00,0.402,0.1580,D#,-16.900,minor,0.0390,0.989,0.1300,0.311,0.196,103.220,4,163.080


We will encode keys into numbers 0-11 and minor and major into 0 and 1, respectively as using oneHotEncoder will introduce more features which is not efficient.

In [12]:
keys= ['C','C#','D','D#', 'E', 'F','F#','G', 'Ab','A','Bb','B' ]
numbers=[x for x in range(12)]
key_dict= dict(zip(keys, numbers))
key_dict.update({'minor':0,'major':1})
df=df_no_year.replace(key_dict)

In [13]:
df.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration
0,0.06,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,126.903
1,0.00,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,98.200
2,0.00,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,181.640
3,0.00,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,176.907
4,0.00,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,163.080


We have a lot of features to work with. Having too much features can lead to overfitting. We will try to reduce the number of features. For that we will see the correlation of each feature with popularity.

In [14]:
df_corr= df.corr()['popularity'].abs().sort_values(ascending=False)
df_corr

popularity          1.000000
acousticness        0.370666
loudness            0.327682
energy              0.302016
instrumentalness    0.236540
danceability        0.187727
time_signature      0.086832
tempo               0.071998
liveness            0.049070
speechiness         0.047311
mode                0.033414
duration            0.027130
key                 0.015163
valence             0.004451
Name: popularity, dtype: float64

In [15]:
df.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration
0,0.06,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3,126.903
1,0.00,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1,98.200
2,0.00,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5,181.640
3,0.00,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3,176.907
4,0.00,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4,163.080


It can be seen that mode, key and valence are less correlated with popularity. So dropping thses features.

In [16]:
least_corr= ['key','mode','valence']
df= df.drop(least_corr, axis=1)
features= df.drop('popularity', axis=1).columns

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587927 entries, 0 to 587926
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        587927 non-null  float64
 1   danceability      587927 non-null  float64
 2   energy            587927 non-null  float64
 3   loudness          587927 non-null  float64
 4   speechiness       587927 non-null  float64
 5   acousticness      587927 non-null  float64
 6   instrumentalness  587927 non-null  float64
 7   liveness          587927 non-null  float64
 8   tempo             587927 non-null  float64
 9   time_signature    587927 non-null  int64  
 10  duration          587927 non-null  float64
dtypes: float64(10), int64(1)
memory usage: 49.3 MB


# Classification Model

We'll try to predict songs' popularities by categorizing popularities(into "high","mid","low"), and classifying songs into those categories.

In [18]:
pd.cut(df['popularity'], bins=3)

0         (-0.001, 0.333]
1         (-0.001, 0.333]
2         (-0.001, 0.333]
3         (-0.001, 0.333]
4         (-0.001, 0.333]
               ...       
587922    (-0.001, 0.333]
587923    (-0.001, 0.333]
587924    (-0.001, 0.333]
587925    (-0.001, 0.333]
587926    (-0.001, 0.333]
Name: popularity, Length: 587927, dtype: category
Categories (3, interval[float64, right]): [(-0.001, 0.333] < (0.333, 0.667] < (0.667, 1.0]]

In [19]:
labels=['low','medium', 'high']
df['popularity']= pd.cut(df['popularity'], bins=3, labels=labels, right=True)

In [20]:
df.popularity.value_counts()

low       363052
medium    213090
high       11785
Name: popularity, dtype: int64

The dataset is unbalanced. This is intuitive, because there aren't as many popular songs as there are non-popular songs. However,imbalance in dataset will tamper with the accuracy of our model. One way to counteract this is by upsampling songs with high popularity. We will then perform K-Nearest Neighbor classification, because they are good at handling noisy data.

Up-sampling songs

In [21]:
df_high= df[df.popularity=='high']
df_mid= df[df.popularity=='medium']
df_low= df[df.popularity=='low']

In [22]:
df_mid_upsampled= resample(df_mid, replace=True, n_samples= 363052, random_state=42)
df_high_upsampled= resample(df_high, replace=True, n_samples= 363052, random_state=42)

list_df_upsampled_tohigh=[df_high_upsampled, df_mid_upsampled, df_low]

df_resampled= pd.concat(list_df_upsampled_tohigh)

In [23]:
df_resampled.popularity.value_counts()

low       363052
medium    363052
high      363052
Name: popularity, dtype: int64

In [24]:
X_re= df_resampled.drop('popularity', axis=1)
y_re= df_resampled['popularity']

In [25]:
X_train, X_test, y_train, y_test= train_test_split(X_re, y_re, random_state= 42, test_size=0.3)

# K-Nearest Neighbors (KNN) Classifier

In [26]:
KNN= KNeighborsClassifier()

KNN.fit(X_train, y_train)

y_pred_classification= KNN.predict(X_test)
print(classification_report(y_test, y_pred_classification))

              precision    recall  f1-score   support

        high       0.88      1.00      0.94    108764
         low       0.70      0.58      0.63    108706
      medium       0.65      0.67      0.66    109277

    accuracy                           0.75    326747
   macro avg       0.74      0.75      0.74    326747
weighted avg       0.74      0.75      0.74    326747



Because KNN is not a tree-based algorithm, it requires standardization. We will put it as part of our function.

In [ ]:
def fit_model(n_neighbors):
    StandardScaler()
    n_neighbors= round(n_neighbors)

    regressor= KNeighborsClassifier(n_neighbors= n_neighbors)
    

    return np.mean(cross_validate(regressor, X_train, y_train, scoring='accuracy', error_score= 'raise', cv=5)['test_score'])

In [29]:
KNN_BO = BayesianOptimization(fit_model,{
        'n_neighbors': (1,80)
    })

KNN_BO.maximize(n_iter=10, init_points=2, allow_duplicate_points=True)

|   iter    |  target   | n_neig... |
-------------------------------------


/var/folders/m6/stmkb2b53fl5wg411y7h4kxr0000gn/T/ipykernel_36592/1916942427.py:5: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  KNN_BO.maximize(n_iter=10, init_points=2, allow_duplicate_points=True)


| 1         | 0.6682    | 16.61     |
| 2         | 0.5787    | 46.93     |
| 3         | 0.6775    | 15.3      |
| 4         | 0.7395    | 3.71      |
| 5         | 0.559     | 80.0      |
| 6         | 0.8104    | 1.0       |
| 7         | 0.6116    | 30.76     |
| 8         | 0.5701    | 63.13     |
| 9         | 0.8104    | 1.311     |
| 10        | 0.5886    | 38.51     |
| 11        | 0.6384    | 23.72     |
| 12        | 0.5654    | 71.44     |


In [30]:
print(KNN_BO.max)

{'target': 0.8103603164973979, 'params': {'n_neighbors': 1.0}}


In [31]:
n_neighbors= KNN_BO.max['params']['n_neighbors']

In [32]:
model = KNeighborsClassifier(n_neighbors= round(n_neighbors))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

        high       0.99      0.96      0.98    112218
         low       0.67      0.81      0.74     89912
      medium       0.83      0.73      0.77    124617

    accuracy                           0.83    326747
   macro avg       0.83      0.83      0.83    326747
weighted avg       0.84      0.83      0.83    326747



In [33]:
RFC= RandomForestClassifier(random_state= random_state)

RFC.fit(X_train, y_train)
y_rfc_pred= RFC.predict(X_test)
print(classification_report(y_rfc_pred, y_test))

              precision    recall  f1-score   support

        high       1.00      0.99      0.99    110127
         low       0.79      0.86      0.82    100271
      medium       0.86      0.81      0.83    116349

    accuracy                           0.88    326747
   macro avg       0.88      0.88      0.88    326747
weighted avg       0.89      0.88      0.88    326747



KNN classifier has a accuracy score of 0.83 (83%) and Random Forest Classifier has score 0.88 (88%).